In [ ]:
# --- Setup: Import Libraries ---
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker

In [ ]:
# --- Create SQLite In-Memory Database ---
DATABASE_URL = "sqlite:///:memory:"  # In-memory SQLite database
engine = create_engine(DATABASE_URL, echo=True)

# Create a Base class for SQLAlchemy ORM
Base = declarative_base()

In [ ]:

# --- Define Employee Table using ORM ---
class Employee(Base):
    __tablename__ = 'employees'

    first = Column(String, primary_key=True)
    last = Column(String, primary_key=True)
    pay = Column(Integer)

    def __init__(self, first, last, pay):
        self.first = first
        self.last = last
        self.pay = pay

    def __repr__(self):
        return f"<Employee(first='{self.first}', last='{self.last}', pay={self.pay})>"

# Create Table
Base.metadata.create_all(engine)

# --- Create a Session ---
Session = sessionmaker(bind=engine)
session = Session()

# --- Insert Employees into Table ---
emp_1 = Employee('John', 'Doe', 80000)
emp_2 = Employee('Jane', 'Doe', 90000)

session.add_all([emp_1, emp_2])
session.commit()

# --- Query: Get Employees by Last Name ---
def get_emps_by_name(lastname):
    return session.query(Employee).filter(Employee.last == lastname).all()

# Fetch employees
emps = get_emps_by_name('Doe')
print("\nEmployees with last name 'Doe':", emps)

# --- Update Employee Pay ---
def update_pay(emp, new_pay):
    emp.pay = new_pay
    session.commit()

update_pay(emp_2, 95000)

# --- Remove an Employee ---
def remove_emp(emp):
    session.delete(emp)
    session.commit()

remove_emp(emp_1)

# Verify Updates
emps = get_emps_by_name('Doe')
print("\nUpdated Employees:", emps)

# --- Load and Execute SQL Query from File ---
query_file = "sql_queries/Q1.txt"
try:
    with open(query_file, "r") as file:
        query = file.read().strip()

    print(f"\nExecuting Query from {query_file}:\n{query}")
    df_query_result = pd.read_sql_query(query, con=engine)
    print("\nQuery Result:")
    print(df_query_result)

except FileNotFoundError:
    print(f"\nError: Query file '{query_file}' not found.")

# --- Append Data from CSV File ---
csv_file = "customers.csv"
try:
    df_csv = pd.read_csv(csv_file)

    # Rename columns to match database schema
    df_csv.rename(columns={
        'Customer Name': 'name',
        'Customer Phone': 'phone_number'
    }, inplace=True)

    # Append to the database
    df_csv.to_sql(name='customers', con=engine, if_exists='append', index=False)
    print(f"\nData from {csv_file} appended to 'customers' table.")

except FileNotFoundError:
    print(f"\nError: CSV file '{csv_file}' not found.")

# Close session
session.close()
